# Conditional Dagster Run: Only When Kaggle CSV Data Changes

This notebook demonstrates how to **only execute the Dagster ELT job when the Kaggle CSV files have changed**.

Pipeline:
1. Load secrets from `.env` (Kaggle & BigQuery).
2. Use `kagglehub` to download Olist CSVs into the Meltano `data/` folder.
3. Compute a checksum for each CSV and compare to the last run, stored in `kaggle_state.json`.
4. **Only if any CSV changed**, trigger the Dagster job that runs Meltano → BigQuery.


In [1]:
!pip install kagglehub


In [2]:
# One-time installs if needed
# !pip install dagster python-dotenv kagglehub

import os
import json
import hashlib
from pathlib import Path

import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter
from dotenv import load_dotenv
from dagster import op, job
import subprocess

# Project paths
PROJECT_ROOT = Path('/home/pingh/Brazilian-E-Commerce-Public-Dataset-by-Olist').resolve()
MELTANO_PROJECT = PROJECT_ROOT / 'meltano_kaggle_csv'
DATA_DIR = MELTANO_PROJECT / 'data'
STATE_PATH = PROJECT_ROOT / 'kaggle_state.json'
ENV_PATH = PROJECT_ROOT / '.env'

print('Project root:', PROJECT_ROOT)
print('Meltano project:', MELTANO_PROJECT)
print('Data dir:', DATA_DIR)
print('State file:', STATE_PATH)

# Load .env
if ENV_PATH.exists():
    load_dotenv(ENV_PATH)
    print('✅ .env loaded')
else:
    print('⚠️ .env not found, make sure it exists at project root')

print('KAGGLE_USERNAME =', os.getenv('KAGGLE_USERNAME'))
print('GCP_PROJECT_ID =', os.getenv('GCP_PROJECT_ID'))


Project root: /home/pingh/Brazilian-E-Commerce-Public-Dataset-by-Olist
Meltano project: /home/pingh/Brazilian-E-Commerce-Public-Dataset-by-Olist/meltano_kaggle_csv
Data dir: /home/pingh/Brazilian-E-Commerce-Public-Dataset-by-Olist/meltano_kaggle_csv/data
State file: /home/pingh/Brazilian-E-Commerce-Public-Dataset-by-Olist/kaggle_state.json
✅ .env loaded
KAGGLE_USERNAME = benben941005
GCP_PROJECT_ID = durable-ripsaw-477914-g0


In [3]:
# Files to download from Kaggle dataset
DATASET_SLUG = 'olistbr/brazilian-ecommerce'
FILES = [
    ('olist_customers', 'olist_customers_dataset.csv'),
    ('olist_geolocation', 'olist_geolocation_dataset.csv'),
    ('olist_order_items', 'olist_order_items_dataset.csv'),
    ('olist_order_payments', 'olist_order_payments_dataset.csv'),
    ('olist_order_reviews', 'olist_order_reviews_dataset.csv'),
    ('olist_orders', 'olist_orders_dataset.csv'),
    ('olist_products', 'olist_products_dataset.csv'),
    ('olist_sellers', 'olist_sellers_dataset.csv'),
    ('product_category_name_translation', 'product_category_name_translation.csv'),
]

def md5_of_file(path: Path) -> str:
    """Return MD5 checksum of a file."""
    h = hashlib.md5()
    with open(path, 'rb') as f:
        for chunk in iter(lambda: f.read(8192), b''):
            h.update(chunk)
    return h.hexdigest()

def load_state() -> dict:
    if STATE_PATH.exists():
        with open(STATE_PATH, 'r') as f:
            return json.load(f)
    return {}

def save_state(state: dict) -> None:
    with open(STATE_PATH, 'w') as f:
        json.dump(state, f, indent=2)

def download_kaggle_and_check_changes() -> bool:
    """Download Kaggle CSVs into Meltano data/ and return True if any file changed.

    Uses MD5 checksums stored in kaggle_state.json to detect changes.
    """
    print('\n📥 Downloading from Kaggle (with change detection)...')
    DATA_DIR.mkdir(parents=True, exist_ok=True)

    previous_state = load_state()
    new_state = {}
    changed = False

    for entity, kaggle_file in FILES:
        print(f"\n=== {kaggle_file} ({entity}) ===")
        df = kagglehub.load_dataset(
            KaggleDatasetAdapter.PANDAS,
            DATASET_SLUG,
            kaggle_file,
        )
        output_path = DATA_DIR / kaggle_file
        df.to_csv(output_path, index=False)
        print('💾 Saved to:', output_path)

        checksum = md5_of_file(output_path)
        new_state[kaggle_file] = checksum
        if previous_state.get(kaggle_file) != checksum:
            changed = True
            print('🔄 Detected change for', kaggle_file)
        else:
            print('✅ No change for', kaggle_file)

    save_state(new_state)
    print('\nChange detected:' , changed)
    return changed


In [4]:
def run_cmd(cmd: str, workdir: Path | None = None) -> None:
    print(f"\n🔹 Running: {cmd}")
    result = subprocess.run(cmd, shell=True, cwd=str(workdir) if workdir else None)
    print('🔹 Return code:', result.returncode)
    if result.returncode != 0:
        raise RuntimeError(f'Command failed: {cmd} (return code {result.returncode})')
    print('✅ Done.\n')

@op
def meltano_to_bigquery_op():
    """Run Meltano CSV → BigQuery pipeline."""
    if not MELTANO_PROJECT.exists():
        raise FileNotFoundError(f'Meltano project not found at {MELTANO_PROJECT}')
    run_cmd('meltano run tap-csv target-bigquery', workdir=MELTANO_PROJECT)
    return 'meltano_done'

@job
def meltano_to_bigquery_job():
    meltano_to_bigquery_op()

print('✅ Dagster job `meltano_to_bigquery_job` defined.')


✅ Dagster job `meltano_to_bigquery_job` defined.


In [5]:
import subprocess
from pathlib import Path

def run_cmd(cmd: str, workdir: Path | None = None) -> None:
    print(f"\n🔹 Running: {cmd}")
    result = subprocess.run(
        cmd,
        shell=True,
        cwd=str(workdir) if workdir else None,
        capture_output=True,
        text=True
    )
    print("🔹 STDOUT:\n", result.stdout)
    print("🔹 STDERR:\n", result.stderr)
    print("🔹 Return code:", result.returncode)
    if result.returncode != 0:
        raise RuntimeError(f"Command failed: {cmd} (return code {result.returncode})")
    print("✅ Done.\n")


In [6]:
result = meltano_to_bigquery_job.execute_in_process()


2025-12-06 18:19:47 +0800 - dagster - DEBUG - meltano_to_bigquery_job - 08df565c-19b0-4725-a05b-70aca7d2adb0 - 18333 - RUN_START - Started execution of run for "meltano_to_bigquery_job".
2025-12-06 18:19:47 +0800 - dagster - DEBUG - meltano_to_bigquery_job - 08df565c-19b0-4725-a05b-70aca7d2adb0 - 18333 - ENGINE_EVENT - Executing steps in process (pid: 18333)
2025-12-06 18:19:47 +0800 - dagster - DEBUG - meltano_to_bigquery_job - 08df565c-19b0-4725-a05b-70aca7d2adb0 - 18333 - meltano_to_bigquery_op - RESOURCE_INIT_STARTED - Starting initialization of resources [io_manager].
2025-12-06 18:19:47 +0800 - dagster - DEBUG - meltano_to_bigquery_job - 08df565c-19b0-4725-a05b-70aca7d2adb0 - 18333 - meltano_to_bigquery_op - RESOURCE_INIT_SUCCESS - Finished initialization of resources [io_manager].
2025-12-06 18:19:47 +0800 - dagster - DEBUG - meltano_to_bigquery_job - 08df565c-19b0-4725-a05b-70aca7d2adb0 - 18333 - LOGS_CAPTURED - Started capturing logs in process (pid: 18333).
2025-12-06 18:19:4


🔹 Running: meltano run tap-csv target-bigquery


2025-12-06 18:42:59 +0800 - dagster - DEBUG - meltano_to_bigquery_job - 08df565c-19b0-4725-a05b-70aca7d2adb0 - 18333 - meltano_to_bigquery_op - STEP_OUTPUT - Yielded output "result" of type "Any". (Type check passed).
2025-12-06 18:42:59 +0800 - dagster - DEBUG - meltano_to_bigquery_job - 08df565c-19b0-4725-a05b-70aca7d2adb0 - 18333 - meltano_to_bigquery_op - HANDLED_OUTPUT - Handled output "result" using IO manager "io_manager"
2025-12-06 18:42:59 +0800 - dagster - DEBUG - meltano_to_bigquery_job - 08df565c-19b0-4725-a05b-70aca7d2adb0 - 18333 - meltano_to_bigquery_op - STEP_SUCCESS - Finished execution of step "meltano_to_bigquery_op" in 23m12s.
/home/pingh/miniconda3/envs/eltn/lib/python3.10/site-packages/dagster/_core/execution/context_creation_job.py:276: RuntimeWarning: coroutine 'BaseEventLoop.shutdown_asyncgens' was never awaited
  pass
2025-12-06 18:42:59 +0800 - dagster - DEBUG - meltano_to_bigquery_job - 08df565c-19b0-4725-a05b-70aca7d2adb0 - 18333 - ENGINE_EVENT - Finished s

🔹 STDOUT:
 
🔹 STDERR:
 2025-12-06T10:19:50.168485Z [warning  ] /home/pingh/miniconda3/envs/eltn/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.18) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)

2025-12-06T10:19:50.336389Z [info     ] Environment 'dev' is active   
2025-12-06T10:19:50.765752Z [info     ] Installing extractor 'tap-csv'
2025-12-06T10:19:50.787731Z [info     ] Installing loader 'target-bigquery'
2025-12-06T10:20:05.172366Z [info     ] Installed extractor 'tap-csv' 
2025-12-06T10:20:18.804285Z [info     ] Installed loader 'target-bigquery'
2025-12-06T10:20:18.804593Z [info     ] Installed 2/2 plugins         
2025-12-06T10:20:19.798655Z [info     ] tap-